jupyter.idigbio.org Pyspark3

In [15]:
sqlContext

In [16]:
index = "idigbio"
nodes = "c18node14.acis.ufl.edu,c18node2.acis.ufl.edu,c18node6.acis.ufl.edu,c18node10.acis.ufl.edu,c18node12.acis.ufl.edu"
query = '{"query": {"bool": {"must": [{"term":{"genus":"acer"}}, {"term":{"stateprovince":"vermont"}}]}}}'
fields = ("uuid,kingdom,phylum,order,class,family,genus,group,specificepithet,infraspecificepithet,scientificname,commonname,canonicalname,highertaxon,"
          "datecollected,continent,country,countrycode,stateprovince,municipality,waterbody,"
          "occurenceid,catalognumber,fieldnumber,collectioncode,collectionid,collector,barcodevalue,basisofrecord,individualcount"
)

fields = "uuid,data.dwc:eventDate"

In [17]:
# Let's try using the API to get a list of column names

import requests

meta_fields_records = (requests
                       .get("http://search.idigbio.org/v2/meta/fields/records")
                       .json()
                       )

field_set = set()
for k,v in meta_fields_records.items():
    if v.get("fieldName", False):
        field_set.add(k)
        
    if k == "data":
        for kd,vd in v.items():
            if vd.get("fieldName", False):
                field_set.add("data.{0}".format(kd))
                pass
        
#print(sorted(fields))
#field_set.remove("geopoint")
#field_set.remove("flags")

#field_list = sorted(field_set)[0:round(len(field_set)/2)]
#field_list = sorted(field_set)[round(len(field_set)/2):]

#field_list = field_list[0:round(len(field_list)/2)]
#field_list = field_list[round(len(field_list)/2):]
# fails too

#field_list = field_list[0:round(len(field_list)/2)]
#field_list = field_list[round(len(field_list)/2):]

#field_list = field_list[0:round(len(field_list)/2)]

fields = ",".join(field_set)

print(fields)

data.dcterms:source,etag,member,institutionid,data.dcterms:license,data.dwc:identificationReferences,data.dwc:highestBiostratigraphicZone,continent,data.dwc:scientificName,data.dwc:higherClassification,data.dwc:waterBody,data.dwc:datasetName,data.dwc:eventID,data.dwc:identifiedBy,data.dwc:maximumElevationInMeters,order,canonicalname,data.fcc:datePicked,version,institutionname,data.dwc:latestEraOrHighestErathem,maxelevation,verbatimeventdate,data.dwc:group,data.dwc:startDayOfYear,data.dwc:dataGeneralizations,data.dwc:acceptedNameUsage,flags,data.dcterms:references,latesteraorhighesterathem,family,scientificname,data.dwc:habitat,data.dwc:rightsHolder,data.dcterms:bibliographicCitation,data.dwc:dynamicProperties,data.dwc:specificEpithet,data.fcc:pickedBy,data.dwc:nameAccordingToID,class,data.dwc:taxonRemarks,data.dwc:maximumDepthInMeters,municipality,data.dwc:earliestAgeOrLowestStage,highertaxon,datemodified,country,data.dwc:collectionID,data.dwc:coordinateUncertaintyInMeters,infraspecifi

In [19]:
bad_field_set = set()
for field in sorted(field_set):
        try:
            df = (sqlContext.read.format("org.elasticsearch.spark.sql")
                .option("es.query", query)
                .option("es.read.field.include", field)
                .option("es.read.as.array.include", "commonnames")
                .option("es.nodes", nodes)
                .load("{0}/records".format(index))
                .write
                .mode("overwrite")
                .parquet("/outputs/scrap")
            )
            print("works: {0}".format(field))
        except:
            print("does not work: {0}".format(field))
            bad_field_set.add(field)


works: barcodevalue
works: basisofrecord
works: bed
works: canonicalname
works: catalognumber
works: class
works: collectioncode
works: collectionid
works: collectionname
works: collector
works: commonname
does not work: commonnames
works: continent
works: coordinateuncertainty
works: country
works: countrycode
works: county
works: data.coreid
works: data.dc:rights
works: data.dcterms:accessRights
works: data.dcterms:bibliographicCitation
works: data.dcterms:language
works: data.dcterms:license
works: data.dcterms:modified
works: data.dcterms:references
works: data.dcterms:rights
works: data.dcterms:rightsHolder
works: data.dcterms:source
works: data.dcterms:type
works: data.dwc:VerbatimEventDate
works: data.dwc:acceptedNameUsage
works: data.dwc:accessRights
works: data.dwc:associatedMedia
works: data.dwc:associatedOccurrences
works: data.dwc:associatedReferences
works: data.dwc:associatedSequences
works: data.dwc:associatedTaxa
works: data.dwc:basisOfRecord
works: data.dwc:bed
works: 

In [20]:
print(bad_field_set)


{'commonnames', 'flags', 'recordids', 'mediarecords'}


In [21]:
field_set -= bad_field_set
fields = ",".join(field_set)
df = (sqlContext.read.format("org.elasticsearch.spark.sql")
    .option("es.query", query)
    .option("es.read.field.include", fields)
    .option("es.nodes", nodes)
    .load("{0}/records".format(index))
)

In [22]:
df.printSchema()
print(df.count())
df.head(10)

root
 |-- barcodevalue: string (nullable = true)
 |-- basisofrecord: string (nullable = true)
 |-- bed: string (nullable = true)
 |-- canonicalname: string (nullable = true)
 |-- catalognumber: string (nullable = true)
 |-- class: string (nullable = true)
 |-- collectioncode: string (nullable = true)
 |-- collectionid: string (nullable = true)
 |-- collectionname: string (nullable = true)
 |-- collector: string (nullable = true)
 |-- commonname: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- coordinateuncertainty: float (nullable = true)
 |-- country: string (nullable = true)
 |-- countrycode: string (nullable = true)
 |-- county: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- coreid: string (nullable = true)
 |    |-- dc:rights: string (nullable = true)
 |    |-- dcterms:accessRights: string (nullable = true)
 |    |-- dcterms:bibliographicCitation: string (nullable = true)
 |    |-- dcterms:language: string (nullable = true)
 |    |-- d

[Row(barcodevalue=None, basisofrecord='preservedspecimen', bed=None, canonicalname='acer dubium', catalognumber='1730298', class='magnoliopsida', collectioncode='botany', collectionid=None, collectionname=None, collector='w. r. adams', commonname=None, continent='north america', coordinateuncertainty=None, country='united states', countrycode='usa', county='chittenden county', data=Row(coreid=None, dc:rights=None, dcterms:accessRights=None, dcterms:bibliographicCitation=None, dcterms:language=None, dcterms:license=None, dcterms:modified=None, dcterms:references=None, dcterms:rights='https://creativecommons.org/publicdomain/zero/1.0/', dcterms:rightsHolder=None, dcterms:source=None, dcterms:type='PhysicalObject', dwc:VerbatimEventDate=None, dwc:acceptedNameUsage=None, dwc:accessRights=None, dwc:associatedMedia=None, dwc:associatedOccurrences=None, dwc:associatedReferences=None, dwc:associatedSequences=None, dwc:associatedTaxa=None, dwc:basisOfRecord='PreservedSpecimen', dwc:bed=None, dw

In [23]:
df.count()

447

In [24]:
df.select(df["uuid"]).head(10)


[Row(uuid='2309622d-a818-4456-b51b-b8bd79587baa'),
 Row(uuid='ef76c868-e5f8-4788-9c31-32947bfdc122'),
 Row(uuid='8536ba01-e79a-4430-b262-3a91305e2df2'),
 Row(uuid='4f8e4b95-5c23-4378-8702-56816b281c51'),
 Row(uuid='3ca004b6-67cc-4fa1-b1b6-40a5e1608b15'),
 Row(uuid='d3ab9e5a-d143-41eb-be2d-780b8b04ea8e'),
 Row(uuid='b7a42fe6-4e67-40ec-9f96-7496e1a59f38'),
 Row(uuid='640f59a8-c1ae-4cbf-9fd7-0b3e9bc3db7d'),
 Row(uuid='adbf7eb7-dc64-4bdc-aa84-7f90e5928b84'),
 Row(uuid='32aa482d-0af5-4ea7-93ae-3035e9c18b35')]

In [25]:
(df.write
.mode("overwrite")
.parquet("/outputs/scrap")
)